# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
! pip install psycopg2-binary --no-cache-dir

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("data/song_data")


In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, orient='columns', lines=True)
df.head(10)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
def get_data(df, columns):
    
    data = df[columns].values.tolist()
    return data


In [9]:
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data1 = df[song_columns].values.tolist()[0]
song_data = get_data(df, song_columns)[0]
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [10]:
song_data1

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [12]:
for song_file in song_files:
    filepath = song_file
    df = pd.read_json(filepath, orient='columns', lines=True)
    print(df)
    song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
    song_data = get_data(df, song_columns)[0]
    cur.execute(song_table_insert, song_data)
    conn.commit()
    
    artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
    artist_data = get_data(df, artist_columns)[0]
    cur.execute(artist_table_insert, artist_data)
    conn.commit()

   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  AR7G5I41187FB4CE6C              NaN               NaN   

   artist_location artist_name             song_id            title  \
0  London, England    Adam Ant  SONHOTT12A8C13493C  Something Girls   

    duration  year  
0  233.40363  1982  
   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  AR8ZCNI1187B9A069B              NaN               NaN   

  artist_location       artist_name             song_id       title  \
0                  Planet P Project  SOIAZJW12AB01853F1  Pink World   

    duration  year  
0  269.81832  1984  
   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARXR32B1187FB57099              NaN               NaN   

  artist_location artist_name             song_id           title   duration  \
0                         Gob  SOFSOCN12A8C143F5D  Face the Ashes  209.60608   

   year  
0  2007  
   num_songs        

   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  AROGWRA122988FEE45              NaN               NaN   

  artist_location      artist_name             song_id        title  \
0                  Christos Dantis  SOSLAVG12A8C13397F  Den Pai Alo   

    duration  year  
0  243.82649     0  
   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARI2JSK1187FB496EF         51.50632          -0.12714   

   artist_location               artist_name             song_id  \
0  London, England  Nick Ingman;Gavyn Wright  SODUJBS12A8C132150   

                  title   duration  year  
0  Wessex Loses a Bride  111.62077     0  
   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  ARD7TVE1187B99BFB1              NaN               NaN   

   artist_location artist_name             song_id    title   duration  year  
0  California - LA      Casual  SOQLGFP12A58A7800E  OAKtown  259.44771     0  
   nu

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_columns = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
artist_data = get_data(df, artist_columns)[0]
artist_data

['ARGCY1Y1187B9A4FA5', 'Gloriana', 'Nashville, TN.', 36.16778, -86.77836]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files("data/log_data")

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, orient='columns', lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [28]:
df = df[df.page == 'NextSong']

df = df[df.userId == '']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [19]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

Series([], Name: ts, dtype: datetime64[ns])

In [20]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('starttime', 'hour', 'day', 'week', 'month', 'year', 'weekday' )

In [21]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,starttime,hour,day,week,month,year,weekday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
df = pd.read_json(filepath, orient='columns', lines = True)
user_columns = ['userId', 'firstName', 'lastName', 'gender', 'level']
user_df = df[user_columns]
user_df = user_df[user_df.userId != '']
user_df.head(50)

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
3,32,Lily,Burns,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
6,92,Ryann,Smith,F,free
7,92,Ryann,Smith,F,free
8,92,Ryann,Smith,F,free
9,92,Ryann,Smith,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None


        
    #insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

InvalidTextRepresentation: invalid input syntax for type integer: ""
LINE 3: ... VALUES ('2018-11-11T10:48:18.796000'::timestamp, '', 'free'...
                                                             ^


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [ ]:
cur.close()